<h1>Capstone Project</h1>

<h3> Scraping Wikipedia Page and adding required content to a data frame

In [ ]:
import pandas as pd

In [4]:
conda install -c anaconda beautifulsoup4 

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.7.1       |           py36_1         143 KB  anaconda
    openssl-1.0.2s             |       h7b6447c_0         3.1 MB  anaconda
    certifi-2019.6.16          |           py36_0         154 KB  anaconda
    soupsieve-1.8              |           py36_0         104 KB  anaconda
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

    soupsieve:      1.8-py36_0        a

In [38]:
from bs4 import BeautifulSoup #importing beautiful soup library

In [25]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' #this is the wikipedia page that contains a list of postal codes in Canada

<h2> Lets start scraping

In [39]:
import requests

In [40]:
page=requests.get(url) #extacting the web information using the get method

In [42]:
soup=BeautifulSoup(page.content,'html.parser') # this line of code parses throgh the extracted page using the html parser.


In [48]:
# These are the three lists that would be used to create a dataframe
postCode=[]
borough=[]  
neighbour=[]

In [49]:
#This loop uses the soup object from the beautiful soup library
#The find method gets us the table and the find_all method looks for all the rows
#We need the row data and that we can find inside the td tag

for row in soup.find('table').find_all('tr'):
    data = row.find_all('td')
    if(len(data) > 0):
        postCode.append(data[0].text)
        borough.append(data[1].text)
        neighbour.append(data[2].text.rstrip('\n')) 

In [51]:
#Here is the data frame
dataF=pd.DataFrame({"PostalCode":postCode,"Borough":borough,"Neighbourhood":neighbour})

In [52]:
dataF.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [53]:
#The question asked to process only those cells that have assigned boroughs and hence this is the code for it 
dataF2=dataF[dataF.Borough!="Not assigned"].reset_index(drop=True)

In [54]:
dataF2.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


<h3>Grouping neighbourhoods with one postal code area </h3>

In [59]:
dataF3 = dataF2.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
dataF3.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [60]:
#If a neighbourhood is not assigned then the neigbourhood will be same as the borough
for index, row in dataF3.iterrows():
    if row["Neighbourhood"] == "Not assigned":
        row["Neighbourhood"] = row["Borough"]
        
dataF3.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [61]:
dataF3.shape

(103, 3)